<a href="https://colab.research.google.com/github/ZikrullaRaxmatov/AI_Pytorch_Keras/blob/main/MNIST_Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Template
1. import libraries
2. dataset load
3. data preparation / procces
4. model create
5. train the model
6. predict (with test dataset)
7. evaluate the model

In [11]:
# import libraries
import numpy as np
import pandas as pd

import torch
from torch.utils.data import DataLoader, Dataset
from torchvision.transforms import transforms

In [12]:
# dataset load
class MNISTDataset(Dataset):
  def __init__(self, csv_file, transform = None):
    self.data = pd.read_csv(csv_file).values
    self.transform = transform

  def __len__(self):
    return len(self.data)

  def __getitem__(self, idx):
    img = self.data[idx, 1:].astype(np.uint8).reshape(28, 28, 1)
    label = self.data[idx, 0].astype(np.int64)
    if self.transform:
      img = self.transform(img)
    return img, label



In [13]:
# data preparation / proccesing
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize(mean=0.5, std=0.5)])

train_dataset = MNISTDataset('./sample_data/mnist_train_small.csv', transform=transform)
test_dataset = MNISTDataset('./sample_data/mnist_test.csv', transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)



In [14]:
train_dataset.data.shape


(19999, 785)

In [15]:
for img, label in train_loader:
  print(img.shape)
  break

torch.Size([64, 1, 28, 28])


In [27]:
#model create
class CNNPytorch(torch.nn.Module):
  def __init__(self, num_classes = 10):
    super(CNNPytorch, self).__init__()
    self.conv1 = torch.nn.Conv2d(1, 32, kernel_size=3)
    self.conv2 = torch.nn.Conv2d(32, 64, kernel_size=3)
    self.pool = torch.nn.MaxPool2d(kernel_size=2)
    self.flatten = torch.nn.Flatten()
    self.fc = torch.nn.Linear(64*5*5, num_classes)

  def forward(self, x):
    x = self.pool(torch.relu(self.conv1(x)))
    x = self.pool(torch.relu(self.conv2(x)))
    x = self.flatten(x)
    x = self.fc(x)
    return torch.softmax(x, dim=1)


model = CNNPytorch()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print(f"Model is running on: {device}")
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)


Model is running on: cpu


In [18]:
model

CNNPytorch(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc): Linear(in_features=1600, out_features=10, bias=True)
)

In [29]:
# train the model
epochs = 5
for epoch in range(epochs):
  model.train()

  running_loss = 0.
  correct = 0
  total = 0

  for epoch_idx, (images, labels) in enumerate(train_loader):
    images, labels = images.to(device), labels.to(device)
    optimizer.zero_grad()
    outputs = model(images)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

    running_loss += loss.item()
    _, predicts = torch.max(outputs, 1)
    total += label.size(0)
    correct += (predicts == labels).sum().item()

    if (epoch_idx + 1) % 312 == 0:
      print(f"Epochs: {epoch+1} / {epochs}")
      print(f"loss: {running_loss/100:.4f} - accuracy: {correct/total:.4f}")

      running_loss = 0.
      correct = 0
      total = 0




Epochs: 1 / 5
loss: 5.3810 - accuracy: 0.7569
Epochs: 2 / 5
loss: 5.3494 - accuracy: 0.7622
Epochs: 3 / 5
loss: 5.3285 - accuracy: 0.7663
Epochs: 4 / 5
loss: 5.3122 - accuracy: 0.7681
Epochs: 5 / 5
loss: 5.3007 - accuracy: 0.7707


In [30]:
# evaluate the model
model.eval()
correct = 0
total = 0
with torch.no_grad():
  for images, labels in test_loader:
    images, labels = images.to(device), labels.to(device)
    outputs = model(images)
    _, predicts = torch.max(outputs, 1)
    total += labels.size(0)
    correct += (predicts == labels).sum().item()

accuracy = correct / total * 100
print(f"Accuracy: {accuracy:.2f} %")


Accuracy: 77.27 %
